In [20]:
from elasticsearch import Elasticsearch, helpers
import csv
import time

# Connexion au cluster Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Nom de l'index
index_name = "reviews"

# Mapping pour l'index
mapping = {
    "mappings": {
        "_meta": {
            "created_by": "yaya"
        },
        "properties": {
            "Company": {"type": "keyword"},
            "Customer": {"type": "keyword"},
            "Date_experience": {"type": "date", "format": "yyyy-MM-dd"},
            "Date_reply": {"type": "date", "format": "yyyy-MM-dd"},
            "Date_review": {"type": "date", "format": "yyyy-MM-dd"},
            "Response_time":{"type": "long"},
            "Experience": {"type": "keyword"},
            "Language": {"type": "keyword"},
            "Number_review": {"type": "long"},
            "Rating": {"type": "long"},
            "Reply": {"type": "keyword"},
            "Status": {"type": "keyword"},
            "Title": {"type": "keyword"},
            "document_id": {"type": "keyword"}
        }
    }
}


def create_elasticsearch_index():
    es = Elasticsearch(hosts="http://localhost:9200")

    # Index des avis clients
    index_name = "reviews"
    # Créer l'index avec le mapping (you can skip this if the index already exists)
    if not es.indices.exists(index=index_name):
        es.indices.create(index=index_name, body=mapping)

    # Lecture du fichier CSV et correction des champs vides
    with open("../data/processed/reviews.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        documents = []
        for idx, row in enumerate(reader):
            if "" in row:
                row["document_id"] = str(idx)  # Ajout de l'ID du document
                for key in list(row.keys()):
                    if key == "":
                        row.pop(key)  # Suppression des champs avec des clés vides
            documents.append(row)

    # Insérer les documents en vrac dans Elasticsearch
    if documents:
        try:
            response = helpers.bulk(es, documents, index=index_name)
            print("Documents insérés avec succès :", response)
        except helpers.BulkIndexError as e:
            print("Erreur lors de l'insertion des documents :")
            for err in e.errors:
                print("Erreur :", err)

    # Vérification des documents insérés
    result = es.search(index=index_name, size=5)  # Récupère les 5 premiers documents

    # Afficher les résultats
    for hit in result["hits"]["hits"]:
        print(hit["_source"])


    # Vérification des documents insérés
    result = es.search(index=index_name, size=5)  # Récupère les 5 premiers documents

    # Afficher les résultats
    for hit in result["hits"]["hits"]:
        print(hit["_source"])

    response = es.search(index="reviews")
    # Récupération du template
    template = es.indices.get_mapping()

    # recherche de tous les documents
    es = Elasticsearch(hosts="http://localhost:9200")

    # Index des avis clients
    index_name = "reviews"
    
    query = {
        "query": {
            "match_all": {}
        }
    }

    # Exécuter la requête
    result = es.search(index="reviews", body=query)

    # Afficher les résultats
    for hit in result["hits"]["hits"]:
        print(hit["_source"])


    # Requête pour obtenir les 5 mots clés positifs
    es = Elasticsearch(hosts="http://localhost:9200")

    # Index des avis clients
    index_name = "reviews"
    query = {
        "query": {
            "match_all": {}
        },
        "aggs": {
            "positive_words": {
                "terms": {
                    "field": "Experience",
                    "size": 5,
                    "min_doc_count": 5
                },
                "aggs": {
                    "rating_filter": {
                        "filter": {
                            "range": {
                                "Rating": {
                                    "gte": 5,
                                    "lte": 5
                                }
                            }
                        }
                    }
                }
            }
        }
    }
    # Exécution de la requête
    result = es.search(index=index_name, body = query)

    # Récupération des résultats
    positive_words = result["aggregations"]["positive_words"]["buckets"]

    for positive_word in positive_words:
        print(f"Mot clé positif : {positive_word['key']} - Nombre d'occurrences : {positive_word['doc_count']}")

    # Recherche des mots les plus récurrents dans les commentaires négatifs
    es = Elasticsearch(hosts="http://localhost:9200")

    # Index des avis clients
    index_name = "reviews"
    query = {
        "query": {
            "match_all": {}
        },
        "aggs": {
            "negative_words": {
                "terms": {
                    "field": "Title",
                    "size": 3,
                    "min_doc_count": 10
                },
                "aggs": {
                    "rating_filter": {
                        "filter": {
                            "range": {
                                "Rating": {
                                    "gte": 1,
                                    "lte": 1
                                }
                            }
                        }
                    }
                }
            }
        }
    }
    # Exécution de la requête
    result = es.search(index=index_name, body=query)

    # Récupération des résultats
    negative_words = result["aggregations"]["negative_words"]["buckets"]

    # Affichage des résultats
    for negative_word in negative_words:
        print(f"Mot clé négatif : {negative_word['key']} - Nombre d'occurrences : {negative_word['doc_count']}")
    
    # Ordonner la sortie du plus petit au plus grand nombre d'occurrences
    negative_words_sorted = sorted(negative_words, key=lambda x: x["doc_count"])

    # Affichage des résultats ordonnés
    for negative_word in negative_words_sorted:
        print(f"Mot clé négatif : {negative_word['key']} - Nombre d'occurrences : {negative_word['doc_count']}")


    # Requête d'agrégation pour obtenir des statistiques sur le champ "Rating"
    es = Elasticsearch(hosts="http://localhost:9200")

    # Index des avis clients
    index_name = "reviews"
    query = {
        "size": 0,
        "aggs": {
            "value_count": {
                "value_count": {
                    "field": "Rating"
                }
            },
            "rating_stats": {
                "stats": {
                    "field": "Rating"
                }
            },
            "extended_ratings_stats": {
                "extended_stats": {
                    "field": "Rating"
                }
            }
        }
    }

    # Exécuter la requête
    result = es.search(index="reviews", body=query)

    # Afficher les résultats de l'agrégation
    aggregations = result.get("aggregations", {})
    value_count = aggregations.get("value_count", {})
    rating_stats = aggregations.get("rating_stats", {})
    extended_ratings_stats = aggregations.get("extended_ratings_stats", {})

    print("Nombre total de valeurs de rating:", value_count.get("value", 0))
    print("Statistiques de rating:")
    print("    Minimum:", rating_stats.get("min", 0))
    print("    Maximum:", rating_stats.get("max", 0))
    print("    Moyenne:", rating_stats.get("avg", 0))
    print("    Somme:", rating_stats.get("sum", 0))
    print("    Écart-type:", extended_ratings_stats.get("std_deviation", 0))
        
    # Liste des entreprises avec le plus de commentaires négatifs
    es = Elasticsearch(hosts="http://localhost:9200")

    # Index des avis clients
    index_name = "reviews"
    query = {
        "query": {
            "match_all": {}
        },
        "sort": [
            {
                "Rating": {
                    "order": "asc"  # Tri ascendant (du plus bas au plus haut)
                }
            }
        ]
    }

    # Exécution de la requête pour récupérer les commentaires triés par note
    result = es.search(index=index_name, body=query)

    # Analyse des commentaires triés pour identifier les entreprises avec le plus de commentaires négatifs
    from collections import Counter

    negative_reviews = []
    for hit in result["hits"]["hits"]:
        rating = int(hit["_source"]["Rating"])  # Convertir la note en entier
        if rating < 3:  # Supposons que 1 à 2 sont considérés comme des notes négatives
            negative_reviews.append(hit["_source"]["Company"])

    # Utilisation de Counter pour compter le nombre de commentaires négatifs par entreprise
    company_count = Counter(negative_reviews)

    # Triez les entreprises par le nombre de commentaires négatifs (du plus au moins)
    sorted_companies = sorted(company_count.items(), key=lambda x: x[1], reverse=True)

    # Afficher les entreprises avec le plus de commentaires négatifs
    for company, count in sorted_companies[:6]:  # Affiche les 5 premières entreprises avec le plus de commentaires négatifs
        print(f"Entreprise : {company}, Nombre de commentaires négatifs : {count}")

    # Le nombre de commentaires non répondues par les entreprises
    # Exécution de la requête
    result = es.search(index=index_name, 
    body={
        "query": {
            "match": {
                "Reply": {
                    "query": "No Reply"
                }
            }
        },
        "aggs": {
            "companies": {
                "terms": {
                    "field": "Company",
                    "size": 10
                }
            }
        }
    })

    # Récupération des résultats
    companies = result["aggregations"]["companies"]["buckets"]

    # Affichage des résultats
    for company in companies:
        print(f"Entreprise : {company['key']} - Nombre de No Reply : {company['doc_count']}")

    # Exécution de la requête pour obtenir le classement des entreprises selon leur note moyenne
    es = Elasticsearch(hosts="http://localhost:9200")

    # Index des avis clients
    index_name = "reviews"
    query = {
        "aggs": {
            "companies": {
                "terms": {
                    "field": "Company"
                },
                "aggs": {
                    "average_rating": {
                        "avg": {
                            "field": "Rating"
                        }
                    }
                }
            }
        }
    }

    result = es.search(index=index_name, body=query)

    # Récupération des résultats
    companies = result["aggregations"]["companies"]["buckets"]

    # Tri des entreprises par leur note moyenne
    companies.sort(key=lambda company: company["average_rating"]["value"], reverse=False)

    # Affichage des résultats
    for company in companies:
        average_ratings = round(company['average_rating']['value'], 2)
        print(f"Entreprise : {company['key']} - Note moyenne : {average_ratings}")

    # Exécution de la requête pour obtenir les différents statuts des commentateurs
    es = Elasticsearch(hosts="http://localhost:9200")

    # Index des avis clients
    index_name = "reviews"
    query = {
        "aggs": {
            "statuses": {
                "terms": {
                    "field": "Status"
                }
            }
        }
    }

    result = es.search(index=index_name, body=query)

    # Récupération des résultats
    statuses = result["aggregations"]["statuses"]["buckets"]

    # Affichage des résultats
    for status in statuses:
        print(f"Status : {status['key']} - Nombre de commentateurs : {status['doc_count']}")

    
    # Exécution de la requête pour obtenir le nombre de types de statut des commentateurs par entreprise
    es = Elasticsearch(hosts="http://localhost:9200")

    # Index des avis clients
    index_name = "reviews"
    query = {
        "aggs": {
            "companies": {
                "terms": {
                    "field": "Company",
                    "size": 10  # Adjust the size based on your needs
                },
                "aggs": {
                    "statuses": {
                        "terms": {
                            "field": "Status"
                        }
                    }
                }
            }
        }
    }

    result = es.search(index=index_name, body=query)

    # Récupération des résultats
    companies = result["aggregations"]["companies"]["buckets"]

    # Affichage des résultats
    for company in companies:
        company_name = company['key']
        statuses = company["statuses"]["buckets"]
        
        print(f"Entreprise : {company_name}")
        for status in statuses:
            status_name = status['key']
            count = status['doc_count']
            print(f"    Status : {status_name} - Nombre de commentateurs : {count}")

    
    # Nombre et Pourcentage de commentaires non répondus par entreprise
    query = {
        "query": {
            "match": {
                "Reply": "No Reply"
            }
        },
        "aggs": {
            "total_no_reply": {
                "value_count": {
                    "field": "Reply"
                }
            },
            "companies": {
                "terms": {
                    "field": "Company"
                },
                "aggs": {
                    "number_of_no_reply": {
                        "value_count": {
                            "field": "Reply"
                        }
                    }
                }
            }
        }
    }

    result = es.search(index=index_name, body=query)
    # Récupération des résultats
    total_no_reply = result["aggregations"]["total_no_reply"]["value"]
    companies = result["aggregations"]["companies"]["buckets"]

    # Affichage des résultats
    for company in companies:
        number_of_no_reply = company["number_of_no_reply"]["value"]
        percentage = (number_of_no_reply / total_no_reply) * 100
        print(f"Entreprise : {company['key']} - Nombre de No Reply : {number_of_no_reply} - Taux de No Reply : {percentage:.2f}%")

    # Exécution de la requête pour obtenir le délai de réponse des entreprises sur les avis ayant obtenu une réponse.
    es = Elasticsearch(hosts="http://localhost:9200")

    # Index des avis clients
    index_name = "reviews"
    query = {
        "aggs": {
            "companies": {
                "terms": {
                    "field": "Company"
                },
                "aggs": {
                    "average_day": {
                        "avg": {
                            "field": "Response_time"
                        }
                    }
                }
            }
        }
    }

    result = es.search(index=index_name, body=query)

    # Récupération des résultats
    companies = result["aggregations"]["companies"]["buckets"]

    # Tri des entreprises par leur note moyenne
    companies.sort(key=lambda company: company["average_day"]["value"], reverse=False)

    # Affichage des résultats
    for company in companies:
        average_days = round(company['average_day']['value'], 2)
        print(f"Entreprise : {company['key']} - Délai de réponse moyen : {average_days}")
    

if __name__ == "__main__":
    create_elasticsearch_index()

"""Verified: Vérifié,
    Invited: Invité,
    Merged: Fusionné,
    Redirected: Redirigé
"""

C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/17282793.py:38: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index_name, body=mapping)


Erreur lors de l'insertion des documents :
Erreur : {'index': {'_index': 'review2', '_id': 'aik1sosB9-HHTkh5OCtR', 'status': 400, 'error': {'type': 'document_parsing_exception', 'reason': "[1:163] failed to parse field [Date_review] of type [date] in document with id 'aik1sosB9-HHTkh5OCtR'. Preview of field's value: '2023-07-28 06:28:34'", 'caused_by': {'type': 'illegal_argument_exception', 'reason': 'failed to parse date field [2023-07-28 06:28:34] with format [yyyy-MM-dd]', 'caused_by': {'type': 'date_time_parse_exception', 'reason': "Text '2023-07-28 06:28:34' could not be parsed, unparsed text found at index 10"}}}, 'data': {'Company': 'Younited Credit', 'Customer': 'M françois GUYOT', 'Number_review': '2', 'Language': 'FR', 'Title': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…', 'Date_review': '2023-07-28 06:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 06:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…SUPER SOCIETE DE CREDI

In [21]:
response = es.search(index="review2")
# Récupération du template
template = es.indices.get_mapping()



In [22]:
# recherche de tous les documents 
query = {
    "query": {
        "match_all": {}
    }
}

# Exécuter la requête
result = es.search(index="review2", body=query)

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])

    

C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/3105312890.py:9: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="review2", body=query)


In [23]:
# recherche de documents contenant "bon produit"
query = {            
    "size": 20,

    "query": {
        "match": {
            "Experience": "bon produit"
        }
    }
}

# Exécuter la requête
result = es.search(index="review", body=query)


# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])
    

{'Company': 'Boursorama Banque', 'Customer': 'FELIX', 'Number_review': '8', 'Language': 'MQ', 'Title': 'EXCELLENT  PRODUIT  FINANCIER', 'Date_review': '2022-11-03 00:00:00', 'Reply': 'No Reply', 'Date_reply': '2022-11-03 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'EXCELLENT  PRODUIT  FINANCIER', 'Date_experience': '2022-11-03 00:00:00', 'document_id': '1865'}
{'Company': 'Boursorama Banque', 'Customer': 'paulo dos reis', 'Number_review': '2', 'Language': 'FR', 'Title': 'nickel', 'Date_review': '2022-05-11 00:00:00', 'Reply': 'No Reply', 'Date_reply': '2022-05-11 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'nickelnickel, super produit', 'Date_experience': '2022-05-11 00:00:00', 'document_id': '2005'}
{'Company': 'Boursorama Banque', 'Customer': 'FELIX', 'Number_review': '8', 'Language': 'MQ', 'Title': 'EXCELLENT  PRODUIT  FINANCIER', 'Date_review': '2022-11-03 00:00:00', 'Reply': 'No Reply', 'Date_reply': '2022-11-03 00:00:00', 'Rating': '5', 'Status':

C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/2055587705.py:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="review", body=query)


In [24]:
from elasticsearch import Elasticsearch

# Connexion à Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Nom de l'index
index_name = "review"

# Expressions à rechercher
expressions = ["bon produit", "bonne qualité", "bon rapport qualité prix", "facile", "top", "excellent"]

# Création de la requête booléenne
query = {
    "size": 20,

    "query": {
        "bool": {
            "should": [  # Utilisation de "should" pour rechercher n'importe laquelle des expressions
                {"match_phrase": {"Reply": expression}} for expression in expressions
            ]
        }
    }
}

# Exécution de la requête pour récupérer les documents contenant l'une des expressions
result = es.search(index=index_name, body=query)

# Afficher les documents correspondants
for hit in result["hits"]["hits"]:
    print(hit["_source"])


C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/1757995309.py:26: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


{'Company': 'Orange Bank', 'Customer': 'Boyer', 'Number_review': '1', 'Language': 'RE', 'Title': 'Orange Bank oh top', 'Date_review': '2022-03-31 00:00:00', 'Reply': 'Bonjour Lorenza,Merci beaucoup pour cet avis au top du top 🙂à très vite,Romain', 'Date_reply': '2022-05-04 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'Orange Bank oh top', 'Date_experience': '2022-03-31 00:00:00', 'document_id': '1472'}
{'Company': 'Orange Bank', 'Customer': 'Boyer', 'Number_review': '1', 'Language': 'RE', 'Title': 'Orange Bank oh top', 'Date_review': '2022-03-31 00:00:00', 'Reply': 'Bonjour Lorenza,Merci beaucoup pour cet avis au top du top 🙂à très vite,Romain', 'Date_reply': '2022-05-04 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'Orange Bank oh top', 'Date_experience': '2022-03-31 00:00:00', 'document_id': '1472'}
{'Company': 'Orange Bank', 'Customer': 'Boyer', 'Number_review': '1', 'Language': 'RE', 'Title': 'Orange Bank oh top', 'Date_review': '2022-03-31 00:00:00',

In [25]:
# Expressions à rechercher
expressions = ["bon produit", "bonne qualité", "bon rapport qualité prix", "facile", "top", "excellent"]

# Création de la requête booléenne
query = {
    "size": 20,

    "query": {
        "bool": {
            "should": [  # Utilisation de "should" pour rechercher n'importe laquelle des expressions
                {"match_phrase": {"Reply": expression}} for expression in expressions
            ]
        }
    }
}

# Exécution de la requête pour récupérer les documents contenant l'une des expressions
result = es.search(index=index_name, body=query)

# Afficher les documents correspondants
for hit in result["hits"]["hits"]:
    print(hit["_source"])


{'Company': 'Orange Bank', 'Customer': 'Boyer', 'Number_review': '1', 'Language': 'RE', 'Title': 'Orange Bank oh top', 'Date_review': '2022-03-31 00:00:00', 'Reply': 'Bonjour Lorenza,Merci beaucoup pour cet avis au top du top 🙂à très vite,Romain', 'Date_reply': '2022-05-04 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'Orange Bank oh top', 'Date_experience': '2022-03-31 00:00:00', 'document_id': '1472'}
{'Company': 'Orange Bank', 'Customer': 'Boyer', 'Number_review': '1', 'Language': 'RE', 'Title': 'Orange Bank oh top', 'Date_review': '2022-03-31 00:00:00', 'Reply': 'Bonjour Lorenza,Merci beaucoup pour cet avis au top du top 🙂à très vite,Romain', 'Date_reply': '2022-05-04 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'Orange Bank oh top', 'Date_experience': '2022-03-31 00:00:00', 'document_id': '1472'}
{'Company': 'Orange Bank', 'Customer': 'Boyer', 'Number_review': '1', 'Language': 'RE', 'Title': 'Orange Bank oh top', 'Date_review': '2022-03-31 00:00:00',

C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/1635953459.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


In [26]:
# Requête de recherche avec l'opérateur or 
query = {
    "size": 20,
    "query": {
        "match": {
        "Experience": {
            "query": "mauvais lent inefficace cher top",
            "operator": "or"
            }
        }
    }
}

# Exécuter la requête
result = es.search(index="review", body=query)

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])

{'Company': 'Younited Credit', 'Customer': 'cliente', 'Number_review': '2', 'Language': 'FR', 'Title': 'Très lent très très lent', 'Date_review': '2023-04-10 00:00:00', 'Reply': "Bonjour,Nous vous remercions d'avoir pris le temps de nous laisser un message.Ayant tenté de vous joindre sans succès, nous tenions à vous faire savoir que nous sommes désolés de constater que vous n'êtes pas pleinement satisfait de nos services. En effet, ayant été victime de notre succès à la suite d’un afflux de demandes, les délais de réponse se sont vus rallongés.Nos équipes mettent tout en œuvre afin de pouvoir à nouveau donner une réponse rapide à l’ensemble de nos clients.Nous prenons en compte vos commentaires dans le cadre de l’amélioration de notre qualité de service.Excellente journée,Tracy", 'Date_reply': '2023-04-12 00:00:00', 'Rating': '2', 'Status': 'Verified', 'Experience': 'Très lent très très lent', 'Date_experience': '2023-04-04 00:00:00', 'document_id': '114'}
{'Company': 'Younited Credit'

C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/3013146843.py:15: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="review", body=query)


In [27]:
# Création de la requête pour récupérer les 5 pires commentaires
query = {
    "size": 5,  # Nombre de commentaires à récupérer
    "sort": [
        {"Rating": {"order": "asc"}}  # Tri par note (ascendant, donc les pires notes d'abord)
    ]
}

# Exécution de la requête pour récupérer les 20 pires commentaires
result = es.search(index=index_name, body=query)

# Afficher les 5 pires commentaires
for hit in result["hits"]["hits"]:
    rating = hit["_source"]["Rating"]
    comment = hit["_source"]["Experience"]
    company = hit["_source"]["Company"]
    print(f"Compagnie: {company}, Note : {rating}, Commentaire : {comment}\n")

C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/4009331916.py:10: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


Compagnie: Younited Credit, Note : 1, Commentaire : UnsatisfiedThe service client is very in professional. Disgusting

Compagnie: Younited Credit, Note : 1, Commentaire : 26 hours and no reply yet26 hours and no reply yet! Why should I trust you? I will definitely go and ask for a credit from another company that cares for its customers! Disappointed.

Compagnie: Younited Credit, Note : 1, Commentaire : Transmission par photo chaotique j’en…Transmission par photo chaotique j’en suis à ma 200 eme photo, scanner à chaque fois refusé soit pas nette, pas cadré grosse perte de temps

Compagnie: Younited Credit, Note : 1, Commentaire : I resigned with FREE and they asked me…I resigned with FREE and they asked me to sent Player to FREE. cannot login your account. and you still taking money from my bank.

Compagnie: Younited Credit, Note : 1, Commentaire : inutileinutile inutile inutile



In [28]:
# Requête de recherche avec filtre de plage de documents avec 4<rating<5
query = {
    "query": {
        "range": {
            "Rating": {
                "gte": 4,
                "lte": 5
            }
        }
    }
}

# Exécuter la requête
result = es.search(index="review", body=query)

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])

C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/1545731499.py:14: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="review", body=query)


{'Company': 'Younited Credit', 'Customer': 'M françois GUYOT', 'Number_review': '2', 'Language': 'FR', 'Title': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…', 'Date_review': '2023-07-28 06:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 06:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'SUPER SOCIETE DE CREDITS TRES REACTIVE…SUPER SOCIETE DE CREDITS TRES REACTIVE sans problêmes merci pour votre pret accorde avec le moins d administratif possible', 'Date_experience': '2023-07-20 00:00:00', 'document_id': '0'}
{'Company': 'Younited Credit', 'Customer': 'Les parisiens', 'Number_review': '2', 'Language': 'FR', 'Title': 'Simple et efficace !', 'Date_review': '2023-07-28 17:28:34', 'Reply': 'No Reply', 'Date_reply': '2023-07-28 17:28:34', 'Rating': '5', 'Status': 'Verified', 'Experience': 'Simple et efficace !', 'Date_experience': '2023-07-19 00:00:00', 'document_id': '1'}
{'Company': 'Younited Credit', 'Customer': 'STEPHANIE RENOUX', 'Number_review': '3', 'Language': 'FR', 'T

In [29]:
# Requête de recherche avec l'opérateur or 
query = {
    "query": {
        "match": {
        "Experience": {
            "query": "simple rapide effice bien top",
            "operator": "or"
            }
        }
    }
}

# Exécuter la requête
result = es.search(index="review", body=query)

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])

C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/1805274503.py:14: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="review", body=query)


{'Company': 'Orange Bank', 'Customer': 'আলী আহমদ সোহান', 'Number_review': '4', 'Language': 'FR', 'Title': 'Qualité', 'Date_review': '2023-05-30 00:00:00', 'Reply': 'Bonjour,Nous vous remercions pour votre retour positif et pour la confiance que vous nous accordez 😉.A bientôt,Roger', 'Date_reply': '2023-06-02 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'QualitéTop du top. Super bien', 'Date_experience': '2023-01-10 00:00:00', 'document_id': '946'}
{'Company': 'Boursorama Banque', 'Customer': 'Hugo G', 'Number_review': '1', 'Language': 'FR', 'Title': 'super bien', 'Date_review': '2022-10-15 00:00:00', 'Reply': 'No Reply', 'Date_reply': '2022-10-15 00:00:00', 'Rating': '5', 'Status': 'Invited', 'Experience': 'super biensuper bien, au top', 'Date_experience': '2022-10-15 00:00:00', 'document_id': '1898'}
{'Company': 'Orange Bank', 'Customer': 'আলী আহমদ সোহান', 'Number_review': '4', 'Language': 'FR', 'Title': 'Qualité', 'Date_review': '2023-05-30 00:00:00', 'Reply': 'Bonjou

In [30]:
from elasticsearch import Elasticsearch

# Connexion au cluster Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Requête d'agrégation pour obtenir des statistiques sur le champ "Rating"
query = {
    "size": 0,
    "aggs": {
        "value_count": {
            "value_count": {
                "field": "Rating"
            }
        },
        "rating_stats": {
            "stats": {
                "field": "Rating"
            }
        },
        "extended_ratings_stats": {
            "extended_stats": {
                "field": "Rating"
            }
        }
    }
}

# Exécuter la requête
result = es.search(index="review", body=query)

# Afficher les résultats de l'agrégation
aggregations = result.get("aggregations", {})
value_count = aggregations.get("value_count", {})
rating_stats = aggregations.get("rating_stats", {})
extended_ratings_stats = aggregations.get("extended_ratings_stats", {})

print("Nombre total de valeurs de rating:", value_count.get("value", 0))
print("Statistiques de rating:")
print("    Minimum:", rating_stats.get("min", 0))
print("    Maximum:", rating_stats.get("max", 0))
print("    Moyenne:", rating_stats.get("avg", 0))
print("    Somme:", rating_stats.get("sum", 0))
print("    Écart-type:", extended_ratings_stats.get("std_deviation", 0))


C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/2341368831.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="review", body=query)


Nombre total de valeurs de rating: 9576
Statistiques de rating:
    Minimum: 1.0
    Maximum: 5.0
    Moyenne: 4.7180451127819545
    Somme: 45180.0
    Écart-type: 0.7912112398310651


In [31]:
# Requête de recherche avec agrégation pour le nombre total de docs pour chaque entreprise
query = {
    "size": 0,
    "aggs": {
        "companies": {
            "terms": {
                "field": "Company",
                "size": 10
            }
        }
    }
}

# Exécuter la requête
result = es.search(index="review", body=query)

# Afficher les résultats de l'agrégation
for bucket in result["aggregations"]["companies"]["buckets"]:
    print(bucket["key"], ":", bucket["doc_count"])

Orange Bank : 3048
Younited Credit : 2776
Boursorama Banque : 1524
Anytime : 948
Cofidis : 948
Floabank : 332


C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/1703345521.py:15: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="review", body=query)


In [32]:
# Création de la requête pour trier les commentaires par note (Rating) de la plus basse à la plus haute
query = {
    "query": {
        "match_all": {}
    },
    "sort": [
        {
            "Rating": {
                "order": "asc"  # Tri ascendant (du plus bas au plus haut)
            }
        }
    ]
}

# Exécution de la requête pour récupérer les commentaires triés par note
result = es.search(index=index_name, body=query, size=5)  # Récupère les 5 premiers commentaires les plus négatifs

# Afficher les résultats
for hit in result["hits"]["hits"]:
    print(hit["_source"])

{'Company': 'Younited Credit', 'Customer': 'HENRY', 'Number_review': '3', 'Language': 'CM', 'Title': 'Unsatisfied', 'Date_review': '2022-08-04 00:00:00', 'Reply': "Bonjour,Nous vous remercions d’avoir pris le temps de nous laisser un message.Nous sommes ravis de vous compter parmi nos clients, toutefois, nous sommes navrés de lire que vous ne soyez pas pleinement satisfait de votre expérience Younited.Nous vous présentons nos excuses pour la gêne occasionnée. Nous sommes toutefois ravis d'avoir pu vous accompagner dans votre projet.N'hésitez pas à nous faire part de d'autres remarques.Excellente journée,\xa0Audrey", 'Date_reply': '2022-09-07 00:00:00', 'Rating': '1', 'Status': 'Verified', 'Experience': 'UnsatisfiedThe service client is very in professional. Disgusting', 'Date_experience': '2022-08-04 00:00:00', 'document_id': '96'}
{'Company': 'Younited Credit', 'Customer': 'Troy', 'Number_review': '1', 'Language': 'FR', 'Title': '26 hours and no reply yet', 'Date_review': '2022-03-17 

C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/1963543287.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query, size=5)  # Récupère les 5 premiers commentaires les plus négatifs


In [33]:
# Liste des entreprises avec le plus de commentaires négatifs
query = {
    "query": {
        "match_all": {}
    },
    "sort": [
        {
            "Rating": {
                "order": "asc"  # Tri ascendant (du plus bas au plus haut)
            }
        }
    ]
}

# Exécution de la requête pour récupérer les commentaires triés par note
result = es.search(index=index_name, body=query)

# Analyse des commentaires triés pour identifier les entreprises avec le plus de commentaires négatifs
from collections import Counter

negative_reviews = []
for hit in result["hits"]["hits"]:
    rating = int(hit["_source"]["Rating"])  # Convertir la note en entier
    if rating < 3:  # Supposons que 1 à 2 sont considérés comme des notes négatives
        negative_reviews.append(hit["_source"]["Company"])

# Utilisation de Counter pour compter le nombre de commentaires négatifs par entreprise
company_count = Counter(negative_reviews)

# Triez les entreprises par le nombre de commentaires négatifs (du plus au moins)
sorted_companies = sorted(company_count.items(), key=lambda x: x[1], reverse=True)

# Afficher les entreprises avec le plus de commentaires négatifs
for company, count in sorted_companies[:6]:  # Affiche les 5 premières entreprises avec le plus de commentaires négatifs
    print(f"Entreprise : {company}, Nombre de commentaires négatifs : {count}")


Entreprise : Younited Credit, Nombre de commentaires négatifs : 6
Entreprise : Orange Bank, Nombre de commentaires négatifs : 3
Entreprise : Cofidis, Nombre de commentaires négatifs : 1


C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/974567915.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


In [34]:
#Mots négatifs les plus fréquents :

from elasticsearch import Elasticsearch
import nltk
from collections import Counter
from nltk.tokenize import word_tokenize

# Connexion à Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Nom de l'index
index_name = "review"

# Récupération des commentaires avec une note négative
query = {
    "query": {
        "range": {
            "Rating": {
                "lt": 3  # Commentaires avec une note inférieure à 3 (supposons que 1 à 2 sont considérés comme des notes négatives)
            }
        }
    }
}

# Exécution de la requête pour récupérer les commentaires
result = es.search(index=index_name, body=query)

# Récupération des commentaires et extraction des mots négatifs
negative_reviews = []
for hit in result["hits"]["hits"]:
    negative_reviews.append(hit["_source"]["Reply"])  # Supposons que les commentaires sont dans le champ "Reply"

# Concaténation de tous les commentaires en un seul texte
all_negative_text = " ".join(negative_reviews)

# Tokenisation des mots
tokens = word_tokenize(all_negative_text)

# Utilisation de Counter pour compter la fréquence des mots
word_freq = Counter(tokens)

# Triez les mots par fréquence (du plus au moins fréquent)
sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

# Afficher les mots négatifs les plus fréquents (par exemple, les 10 premiers)
for word, freq in sorted_words[:30]:
    print(f"Mot : {word}, Fréquence : {freq}")


C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/2177682733.py:26: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


Mot : de, Fréquence : 38
Mot : ,, Fréquence : 18
Mot : vous, Fréquence : 17
Mot : ., Fréquence : 14
Mot : à, Fréquence : 13
Mot : nous, Fréquence : 10
Mot : votre, Fréquence : 10
Mot : pas, Fréquence : 8
Mot : la, Fréquence : 8
Mot : nos, Fréquence : 7
Mot : Bonjour, Fréquence : 6
Mot : ’, Fréquence : 6
Mot : le, Fréquence : 6
Mot : que, Fréquence : 6
Mot : pour, Fréquence : 6
Mot : un, Fréquence : 5
Mot : sommes, Fréquence : 5
Mot : Younited, Fréquence : 5
Mot : Je, Fréquence : 5
Mot : Nous, Fréquence : 4
Mot : ne, Fréquence : 4
Mot : journée, Fréquence : 4
Mot : en, Fréquence : 4
Mot : No, Fréquence : 4
Mot : Reply, Fréquence : 4
Mot : remercions, Fréquence : 3
Mot : d, Fréquence : 3
Mot : pris, Fréquence : 3
Mot : temps, Fréquence : 3
Mot : laisser, Fréquence : 3


In [35]:
#Mots négatifs les plus fréquents superieurs à 5 lettres et le commentaires

from elasticsearch import Elasticsearch
import nltk
from collections import Counter
from nltk.tokenize import word_tokenize

# Connexion à Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Nom de l'index
index_name = "review"

# Récupération des commentaires avec une note négative
query = {
    "query": {
        "range": {
            "Rating": {
                "lt": 2  # Commentaires avec une note inférieure à 2 
            }
        }
    }
}

# Exécution de la requête pour récupérer les commentaires
result = es.search(index=index_name, body=query)

# Récupération des commentaires et extraction des mots négatifs
negative_reviews = []
for hit in result["hits"]["hits"]:
    negative_reviews.append(hit["_source"]["Reply"])  # Supposons que les commentaires sont dans le champ "Reply"

# Concaténation de tous les commentaires en un seul texte
all_negative_text = " ".join(negative_reviews)

# Tokenisation des mots
tokens = word_tokenize(all_negative_text)

# Filtrez les mots pour qu'ils aient plus de 5 lettres
filtered_words = [word for word in tokens if len(word) > 5]

# Utilisation de Counter pour compter la fréquence des mots
word_freq = Counter(filtered_words)

# Triez les mots par fréquence (du plus au moins fréquent)
sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

# Afficher les mots négatifs les plus fréquents (par exemple, les 10 premiers)
for word, freq in sorted_words[:30]:
    
    print(f"Mot : {word}, Fréquence : {freq}")


C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/1887110308.py:26: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


Mot : Bonjour, Fréquence : 6
Mot : sommes, Fréquence : 5
Mot : Younited, Fréquence : 5
Mot : journée, Fréquence : 3
Mot : Credit, Fréquence : 3
Mot : pouvez, Fréquence : 3
Mot : remercions, Fréquence : 2
Mot : laisser, Fréquence : 2
Mot : message.Nous, Fréquence : 2
Mot : toutefois, Fréquence : 2
Mot : navrés, Fréquence : 2
Mot : pleinement, Fréquence : 2
Mot : satisfait, Fréquence : 2
Mot : expérience, Fréquence : 2
Mot : présentons, Fréquence : 2
Mot : excuses, Fréquence : 2
Mot : occasionnée, Fréquence : 2
Mot : d'avoir, Fréquence : 2
Mot : d'autres, Fréquence : 2
Mot : remarques.Excellente, Fréquence : 2
Mot : France, Fréquence : 2
Mot : auprès, Fréquence : 2
Mot : commentaire, Fréquence : 2
Mot : l'équipe, Fréquence : 2
Mot : contacter, Fréquence : 2
Mot : webmaster, Fréquence : 2
Mot : compter, Fréquence : 1
Mot : clients, Fréquence : 1
Mot : Younited.Nous, Fréquence : 1
Mot : accompagner, Fréquence : 1


In [36]:
# 10 Mots positifs les plus fréquents superieurs à 5 lettres et le commentaires
from elasticsearch import Elasticsearch

# Connexion à Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Nom de l'index
index_name = "review"

# Création de la requête pour rechercher les commentaires positifs
query = {
    "query": {
        "range": {
            "Rating": {
                "gte": 4  # Supposons que 4 et 5 sont considérés comme des notes positives
            }
        }
    }
}

# Exécution de la requête pour récupérer les commentaires positifs
result = es.search(index=index_name, body=query)

# Récupération des commentaires positifs
positive_reviews = [hit["_source"]["Reply"] for hit in result["hits"]["hits"]]

# Concaténation de tous les commentaires en un seul texte
all_positive_text = " ".join(positive_reviews)

# Tokenisation des mots
from nltk.tokenize import word_tokenize
tokens = word_tokenize(all_positive_text)

# Filtre pour les mots de plus de 5 lettres
filtered_tokens = [token for token in tokens if len(token) > 5]

# Utilisation de Counter pour compter la fréquence des mots
from collections import Counter
word_freq = Counter(filtered_tokens)

# Triez les mots par fréquence (du plus au moins fréquent)
sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

# Afficher les 10 mots les plus fréquents
top_10_words = sorted_words[:20]

# Afficher les mots et leurs fréquences ainsi que les commentaires correspondants
for word, freq in top_10_words:
    print(f"Mot : {word}, Fréquence : {freq}")
    # Recherche des commentaires contenant ce mot
    matching_comments = [review for review in positive_reviews if word in review]
    # Afficher les commentaires correspondants
    for comment in matching_comments[:5]:  # Afficher les 5 premiers commentaires
        print(f"Commentaire : {comment}\n")


C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/608767723.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


Mot : remercions, Fréquence : 10
Commentaire : Bonjour,Nous vous remercions d’avoir pris le temps de nous laisser un message et sommes ravis de lire que vous êtes pleinement satisfait de votre expérience Younited.Nous vous remercions une nouvelle fois pour votre confiance.Excellente journée,Alexandra

Commentaire : Bonjour,Nous vous remercions d’avoir pris le temps de nous laisser un message et sommes ravis de lire que vous êtes pleinement satisfait de votre expérience Younited.Nous vous remercions une nouvelle fois pour votre confiance.Excellente journée,Alexandra

Commentaire : Bonjour, Nous vous remercions pour votre commentaire et sommes navrés de lire que vous n'êtes pas pleinement satisfait de votre expérience Younited.Le taux d’intérêt dépend du montant emprunté, de la durée, et de votre situation personnelle que vous avez renseigné lors de la simulation de votre demande crédit.Chez Younited, nous nous efforçons de proposer des financements à des taux compétitifs afin de pouvoir

In [37]:
# la liste des entreprises avec le plus de "No Reply" dans la colonne Reply

from elasticsearch import Elasticsearch

# Connexion à Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Nom de l'index
index_name = "review"

# Création de la requête pour agréger les entreprises avec le plus de "No Reply"
query = {
    "size": 0,  # Ne pas récupérer les documents, uniquement les agrégations
    "aggs": {
        "companies_with_no_reply": {
            "terms": {
                "field": "Company.keyword",  # Champ de l'entreprise
                "size": 6  # Nombre d'entreprises à afficher (top 10)
            },
            "aggs": {
                "no_reply_count": {
                    "filter": {
                        "term": {"Reply": "No Reply"}  # Filtre pour les "No Reply"
                    }
                }
            }
        }
    }
}

# Exécution de la requête pour agréger les entreprises avec le plus de "No Reply"
result = es.search(index=index_name, body=query)

# Récupération des agrégations
aggregations = result["aggregations"]["companies_with_no_reply"]["buckets"]

# Afficher les entreprises avec le plus de "No Reply"
for bucket in aggregations:
    company_name = bucket["key"]
    no_reply_count = bucket["no_reply_count"]["doc_count"]
    print(f"Entreprise : {company_name}, Nombre de 'No Reply' : {no_reply_count}")


C:\Users\yaya-\AppData\Local\Temp/ipykernel_9892/4140706961.py:32: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index_name, body=query)


In [38]:
from elasticsearch import Elasticsearch

# Connexion à Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")

# Nom de l'index
index_name = "review3"

# Extraction des mots clés négatifs
def get_negative_keywords():
    query = {
        "query": {
            "range": {
                "Rating": {
                    "lt": 3  # Filtrer les commentaires avec une note inférieure à 3 (négatifs)
                }
            }
        }
    }
    aggs = {
        "top_negative_keywords": {
            "terms": {
                "field": "Reply",
                "size": 5
            }
        }
    }
    result = es.search(index="review2", size=0, body={"query": query, "aggs": aggs})
    return [bucket["key"] for bucket in result["aggregations"]["top_negative_keywords"]["buckets"]]

# Extraction des mots clés positifs
def get_positive_keywords():
    query = {
        "query": {
            "range": {
                "Rating": {
                    "gte": 4  # Filtrer les commentaires avec une note de 4 ou plus (positifs)
                }
            }
        }
    }
    aggs = {
        "top_positive_keywords": {
            "terms": {
                "field": "Reply",
                "size": 5
            }
        }
    }
    result = es.search(index="review2", size=0, body={"query": query, "aggs": aggs})
    return [bucket["key"] for bucket in result["aggregations"]["top_positive_keywords"]["buckets"]]

# Entreprises avec le plus de "No Reply"
def get_companies_with_most_no_reply():
    query = {
        "query": {
            "term": {
                "Reply": "No Reply"
            }
        }
    }
    aggs = {
        "companies_with_most_no_reply": {
            "terms": {
                "field": "Company",
                "size": 5
            }
        }
    }
    result = es.search(index="review2", size=0, body={"query": query, "aggs": aggs})
    return [bucket["key"] for bucket in result["aggregations"]["companies_with_most_no_reply"]["buckets"]]

# Classement des entreprises selon leur note moyenne
def get_company_average_ratings():
    aggs = {
        "company_average_ratings": {
            "terms": {
                "field": "Company",
                "size": 5
            },
            "aggs": {
                "avg_rating": {
                    "avg": {
                        "field": "Rating"
                    }
                }
            }
        }
    }
    result = es.search(index="review2", size=0, body={"aggs": aggs})
    return [{
        "Company": bucket["key"],
        "AverageRating": bucket["avg_rating"]["value"]
    } for bucket in result["aggregations"]["company_average_ratings"]["buckets"]]

# Groupe des commentateurs selon le "status"
def get_commentator_groups_by_status():
    aggs = {
        "commentator_groups": {
            "terms": {
                "field": "Status",
                "size": 5
            }
        }
    }
    result = es.search(index="review2", size=0, body={"aggs": aggs})
    return [bucket["key"] for bucket in result["aggregations"]["commentator_groups"]["buckets"]]

# Appel des fonctions pour répondre aux questions
if __name__ == "__main":
    negative_keywords = get_negative_keywords()
    print("Mots clés négatifs les plus utilisés :", negative_keywords)

    positive_keywords = get_positive_keywords()
    print("Mots clés positifs les plus utilisés :", positive_keywords)

    companies_with_most_no_reply = get_companies_with_most_no_reply()
    print("Entreprises avec le plus de 'No Reply' :", companies_with_most_no_reply)

    company_average_ratings = get_company_average_ratings()
    print("Classement des entreprises selon leur note moyenne :", company_average_ratings)

    commentator_groups = get_commentator_groups_by_status()
    print("Groupe des commentateurs selon le 'status' :", commentator_groups)

